In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import re 
import string

In [2]:
df=pd.read_csv('fake_or_real_news.csv')

In [3]:
df

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [4]:
df['fake']=df['label'].apply(lambda x:0 if x == "REAL" else 1)

In [5]:
data =df.drop("label", axis=1)

In [6]:
data

,id,title,text,fake
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,0
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,1
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,1
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",0


In [7]:
x,y=data['text'],data['fake']

In [8]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [9]:
x_train

896     The Iowa caucuses are 11 weeks away. That is a...
2120     Does The U.S. Government Really Know Who Hack...
5316    In the time since Justice Antonin Scalia’s pas...
4183    0 Add Comment \nA GROUP of over a thousand civ...
4744                                                     
                              ...                        
1288    BREAKING : Trump BEATING “Federal Investigatio...
560     Genetically Modified Crops in U.S. Fail to Del...
2030    Dakota Access pipeline protesters occupy Hilla...
473     HAMPTON, N.H. -- Hillary Rodham Clinton said F...
1104    Lawyer Who Kept Hillary Campaign Chief Out of ...
Name: text, Length: 5068, dtype: object

In [10]:
Vectorizer=TfidfVectorizer(stop_words='english',max_df=0.7)
x_train_vectorizer=Vectorizer.fit_transform(x_train)
x_test_vectorizer=Vectorizer.transform(x_test)

In [11]:
clf= LinearSVC()
clf.fit(x_train_vectorizer,y_train)

LinearSVC()

In [12]:
pred_clf=clf.predict(x_train_vectorizer)
clf.score(x_test_vectorizer,y_test)

0.9329123914759274

In [13]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(x_train_vectorizer,y_train)

LogisticRegression()

In [14]:
pred_lr=lr.predict(x_train_vectorizer)
lr.score(x_test_vectorizer,y_test)

0.9179163378058406

In [15]:
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier()
DT.fit(x_train_vectorizer,y_train)

DecisionTreeClassifier()

In [16]:
pred_DT=lr.predict(x_train_vectorizer)
DT.score(x_test_vectorizer,y_test)

0.8176795580110497

In [25]:
def wordopt(text):
    text=text.lower()
    text=re.sub('\[,*?\]','',text)
    text=re.sub("\\W","",text)
    text=re.sub('https?://\S+|www\.\S+','',text)
    text=re.sub('<,*?>+','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    return text

In [26]:
df['text']= df['text'].apply(wordopt)

In [42]:
def out_lable(n):
    if n==1:
        return "Fake News"
    elif n==0:
        return "Not A Fake News"
def manual_testing(news):
    testing_news={"text":[news]}
    new_def_test=pd.DataFrame(testing_news)
    new_def_test["text"]=new_def_test["text"].apply(wordopt)
    new_x_test=new_def_test["text"]
    new_xv_test=Vectorizer.transform(new_x_test)
    pred_clf=clf.predict(new_xv_test)
    pred_lr=lr.predict(new_xv_test)
    pred_DT=DT.predict(new_xv_test)
    
    return print("\n\nclf Prediction: {} \nlr Prediction: {} \nDT Prediction: {}".format(out_lable(pred_clf[0]),out_lable(pred_lr[0]),out_lable(pred_DT[0])))
    
    

In [43]:
news=str(input())
manual_testing(news)

Women give birth to 17 identical babies in a single pregnancy.


clf Prediction: Fake News 
lr Prediction: Fake News 
DT Prediction: Fake News
